In [129]:
import numpy as np
import pandas as pd
import torch.distributions as dst
import scipy.stats as st

In [220]:
def sample_uniform(n_samples=1000):
    """
    Sample from a Uniform(-1,1) distribution

    Parameters:
    n_samples - number of samples to draw
    """
    return dst.Uniform(-1,1).sample((n_samples,)).numpy()

def sample_normal(mu, std_dev, n_samples=1000):
    """
    Sample from a distribution of N(mu,std_dev)

    Parameters:
    mu - sampled mean from a Uniform(-1,1)
    std_dev - the known standard deviation from the Normal distribution
    n_samples - number of samples to draw from the distribution
    
    """
    return dst.Normal(loc=mu, scale=std_dev).sample((1000,)).numpy()

def get_confidence_interval(normal, z_val, std_dev, n_samples):
    """
    Get confidence interval for accepted mu samples
    
    Parameters:
    normal - sample from a normal distribution
    z_val - given z-score from the hypothesis test
    std_dev - the known standard deviation from the normal distribution from which we sample
    n_samples - number of samples in the normal sample
    """
    
    return (np.mean(normal) - (z_val*std_dev/np.sqrt(n_samples)), np.mean(normal) + (z_val*std_dev/np.sqrt(n_samples)))

def hypothesis_test(alpha_level, n_samples=1000, n_normals=1000, std_dev=1):
    """
    Run a hypothesis test of
    H0: mu >= 0 vs.
    H1: mu < 0 

    with provided significance level alpha. We sample possible mu-s from
    a Uniform(-1,1) and then get 1000 samples from a N(mu,1) distribution
    and see the % of how often mu falls into the confidence inteval

    Parameters:
    alpha_level - significance level of the null hypothesis
    n_samples - number of samples to get from the normal distribution
    n_normals - number of normals to generate samples from per mu
    std_dev - the "known" standard deviation for the normal distribution
    """
    z_val = st.norm.ppf(1-(alpha_level/2))
    mu_array = sample_uniform(n_samples=20)
    count_mu_in_confidence = 0
    count_null_accepted = 0
   
    total = 0

    for mu in mu_array:
       
        for normal in [sample_normal(mu=mu, std_dev=std_dev, n_samples=n_samples) for _ in range(n_normals)]:
            total += 1
            # acceptance criterion for null hypothesis
        
            if st.norm.cdf( np.mean(normal)/(std_dev/np.sqrt(n_samples))) <= (1- alpha_level):
                count_null_accepted += 1
                confidence_interval = get_confidence_interval(normal=normal,
                                                              z_val=z_val,
                                                              std_dev=std_dev,
                                                              n_samples=n_samples)
                
                if confidence_interval[0] <= mu <= confidence_interval[1]:
                    count_mu_in_confidence += 1
    print("mu ~ Uniform(-1, 1)")
    print("Sample from N(mu, 1)\n")
    print("H0 : mu >= 0\nH1 : mu < 0\n")
    print(f"Significance level: {alpha_level}\n")
    print("Accepted ratio: ", count_null_accepted/total)
    print(f"Confidence Interval contains mu sample: {round(100*count_mu_in_confidence/count_null_accepted,2)}%\n")
    print("------------------------------------------------------------------------------")


In [221]:
for i in range(1,3):  
    hypothesis_test(10**(-i), n_normals=100, n_samples=1000)




mu ~ Uniform(-1, 1)
Sample from N(mu, 1)

H0 : mu >= 0
H1 : mu < 0

Significance level: 0.1

Accepted ratio:  0.5995
Confidence Interval contains mu sample: 90.58%

------------------------------------------------------------------------------
mu ~ Uniform(-1, 1)
Sample from N(mu, 1)

H0 : mu >= 0
H1 : mu < 0

Significance level: 0.01

Accepted ratio:  0.4
Confidence Interval contains mu sample: 98.88%

------------------------------------------------------------------------------


In [222]:
hypothesis_test(0.05, n_normals=50)
hypothesis_test(0.025, n_normals=50)

mu ~ Uniform(-1, 1)
Sample from N(mu, 1)

H0 : mu >= 0
H1 : mu < 0

Significance level: 0.05

Accepted ratio:  0.488
Confidence Interval contains mu sample: 94.47%

------------------------------------------------------------------------------
mu ~ Uniform(-1, 1)
Sample from N(mu, 1)

H0 : mu >= 0
H1 : mu < 0

Significance level: 0.025

Accepted ratio:  0.525
Confidence Interval contains mu sample: 96.95%

------------------------------------------------------------------------------
